In [12]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, Column, Integer, String, JSON, DateTime
from datetime import timedelta, datetime

In [13]:
%load_ext sql
%sql sqlite:///database/user_presets.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [14]:
%%sql
SELECT * 
FROM sqlite_schema

 * sqlite:///database/user_presets.db
Done.


type,name,tbl_name,rootpage,sql
table,profiles,profiles,2,"CREATE TABLE profiles ( id INTEGER NOT NULL, user_name VARCHAR, profile_name VARCHAR, settings JSON, ""timestamp_UTC"" DATETIME, PRIMARY KEY (id), UNIQUE (id))"
index,sqlite_autoindex_profiles_1,profiles,3,None


In [15]:
%%sql
SELECT name
FROM sqlite_schema
WHERE type = "table"

 * sqlite:///database/user_presets.db
Done.


name
profiles


In [16]:
%%sql
select *
from users
;

 * sqlite:///database/user_presets.db
(sqlite3.OperationalError) no such table: users
[SQL: select *
from users
;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [17]:
%%sql
select *
from profiles
;

 * sqlite:///database/user_presets.db
Done.


id,user_name,profile_name,settings,timestamp_UTC
1,Luke,Luke_test_,"{""time_line"": ""7 Days"", ""start_date"": ""2023-11-06 12:15:58.395009"", ""end_date"": ""2023-11-21 12:15:58.395009"", ""media_buyer"": ""Alice"", ""active_days"": 15, ""campaign"": ""AP_Aug14_Lawyers_T3_NEW_WWBL""}",2023-11-21 11:15:58.527492


In [56]:
engine = create_engine('sqlite:///database/user_presets.db')

# Your SQL query
query = "SELECT * FROM profiles"

# Execute the query and fetch the results into a Pandas DataFrame
df = pd.read_sql_query(query, engine)
df["settings"][9]

'{"time_line": "Lifetime", "start_date": "2023-11-21 12:52:12.393743", "end_date": "2023-11-01 12:52:12.393743", "media_buyer": "Alice", "active_days": 20, "campaign": "AP_Aug14_Lawyers_T3_NEW_WWBL"}'

In [45]:
Base = declarative_base()
class Profile(Base):
    """
    SQLAlchemy model for the 'profiles' table.

    Attributes:
        id (int): Unique identifier for each profile.
        user_name (str): User name associated with the profile.
        profile_name (str): Name of the profile.
        settings (dict): JSON representation of profile settings.
        timestamp_UTC (datetime): Timestamp indicating when the profile was created (in UTC).

    Note:
        The 'timestamp_UTC' field is automatically populated with the current UTC time when a new profile is created.

    Example:
        To create a new profile instance:
        >>> profile = Profile(user_name='JohnDoe', profile_name='Default', settings={'option1': True, 'option2': 'value'})
    """

    __tablename__ = 'profiles'

    id = Column(Integer, unique=True, primary_key=True)
    user_name = Column(String)
    profile_name = Column(String)
    settings = Column(JSON)
    timestamp_UTC = Column(DateTime, default=datetime.utcnow)  # Default to the current UTC time


/var/folders/_b/_rc6tjkn42n6t50xp7rhttps9db1hv/T/ipykernel_5970/3076849795.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [46]:
# Create an SQLAlchemy engine with a SQLite database file named 'user_presets.db' in the 'database' folder
engine = create_engine('sqlite:///database/user_presets.db')

# Create the database tables defined in the Base class (assuming Base is a declarative_base())
# and bind them to the engine
# Base.metadata.create_all(bind=engine)

# Create a Session class bound to the engine
Session = sessionmaker(bind=engine)
def load_profile(user_name):
    session = Session()
    # Query the database to retrieve all profiles for the specified user
    profiles = session.query(Profile).filter_by(user_name=user_name).all()

    # Create a list of dictionaries containing 'user_name', 'profile_name', 'settings', and 'timestamp_UTC' for each profile
    profile_data = [
        {
            'user_name': user_name,
            'profile_name': profile.profile_name,
            'settings': profile.settings,
            'timestamp_UTC': profile.timestamp_UTC
        }
        for profile in profiles
    ]
    
    df = pd.DataFrame(profile_data)
    return df 

In [52]:
df = load_profile(user_name="Alice")

In [53]:
df

,user_name,profile_name,settings,timestamp_UTC
0,Alice,Alice_test_,"{'time_line': '7 Days', 'start_date': '2023-10...",2023-11-21 11:26:19.325723
1,Alice,Alice_test_2,"{'time_line': '7 Days', 'start_date': '2023-10...",2023-11-21 11:32:52.580235


In [35]:
saved_settings = df[(df["user_name"]=="Luke") & (df["profile_name"]=="Luke_test_")]['settings']

In [36]:
saved_settings.values[0]

{'time_line': '7 Days',
 'start_date': '2023-11-06 12:15:58.395009',
 'end_date': '2023-11-21 12:15:58.395009',
 'media_buyer': 'Alice',
 'active_days': 15,
 'campaign': 'AP_Aug14_Lawyers_T3_NEW_WWBL'}

In [38]:
result_dict = saved_settings.values[0]
result_dict['start_date'].split(' ')[0].replace('-', '/')

'2023/11/06'

In [39]:
# import json
# result_dict = json.loads(saved_settings[0])

# # Print the resulting dictionary
# print(result_dict)

In [40]:
result_dict["time_line"]

'7 Days'

In [41]:
result_dict["media_buyer"]

'Alice'

In [42]:
result_dict['active_days']

15

In [43]:
result_dict["campaign"]

'AP_Aug14_Lawyers_T3_NEW_WWBL'